# 本コードについて
[polars](https://github.com/pola-rs/polars)ライブラリを使って、ユーザ一覧、ユーザ操作一覧、リアクション一覧を集計するサンプル

幾つかの例を踏まえつつ、ユーザごとの活動度(actions.point + reactions.reward)のランキングを出すことを目標とする

## 更新履歴

- 2022.03.18 polars v0.16.14で動作を検証
  - sortのreverse属性がdescending属性にリネーム
  - with_columnがwith_columnsにリネーム
- 2022.12.30 polars v0.15.8で動作を検証、datetimeのタイムゾーンがprint時に表示される旨を追記
- 2022.09.17 polars v0.14.0および処理上の警告に対応

## データの関係
```
< users > (親)
| --- < actions > (子) 
  | --- < reactions > (孫)
```

## データ構造
- users.csv: ユーザ一覧
  - id: ユーザID。常にユニークな文字列
  - name: ユーザ名
  - email: ユーザのemailアドレス
  - type: ユーザの種類。personalは個人で、enterpriseは営利団体としている
  - leaved: 退会済みかどうか
- actions.csv: ユーザ操作一覧
  - id: アクションID。常にユニークな文字列
  - user_id: ユーザ操作をしたユーザのID
  - date: 操作日時 ※
  - message: 操作に関するメッセージ
  - point: 操作により得られたポイント。整数値 or 欠損値
- reactions.csv: ユーザ操作に対するリアクション一覧
  - id: リアクションID
  - action_id: リアクション対象のアクションID
  - user_id: リアクション元のユーザID
  - date: リアクション日時 ※
  - type: リアクションの種類
  - reward: リアクションによる受け取りポイント
  
※ いずれもISO8601フォーマットの文字列とする

In [1]:
import numpy as np
import polars as pl

# ユーザデータの解析

In [2]:
df_users = pl.read_csv("users.csv")
df_users

id,name,email,type,leaved
str,str,str,str,bool
"""BcHgeZkTsc""","""アリス""","""hoge@hoge.exam...","""personal""",false
"""KHPiabVr3o""","""ボブ""","""fuga@fuga.org""","""personal""",false
"""AQA7LkexXv""","""チャーリー""","""foo@foo.net""","""personal""",true
"""C82ZQKSQk7""","""ダービー""","""baz@baz.biz""","""enterprise""",false
"""HzZow64HGH""","""エリザベス""","""bar@bar.info""","""enterprise""",false


## idとnameを取り出す
以下の様に、listで必要な列を指定するか、スライスする

- listの場合、列が隣り合ってなくてもよい
- pandasと異なり、indexが存在しないのでpolars.DataFrame.locは存在しない。スライスが等価

In [3]:
df_users[["id", "name"]]

id,name
str,str
"""BcHgeZkTsc""","""アリス"""
"""KHPiabVr3o""","""ボブ"""
"""AQA7LkexXv""","""チャーリー"""
"""C82ZQKSQk7""","""ダービー"""
"""HzZow64HGH""","""エリザベス"""


In [4]:
df_users[:, :"name"]    # same as df_users.loc[:, :"name"] of pd.DataFrame

id,name
str,str
"""BcHgeZkTsc""","""アリス"""
"""KHPiabVr3o""","""ボブ"""
"""AQA7LkexXv""","""チャーリー"""
"""C82ZQKSQk7""","""ダービー"""
"""HzZow64HGH""","""エリザベス"""


## enterprise ユーザ or 退会済みユーザを取り出す: pl.DataFrame.filter等
複数条件指定時、ANDは"&"、ORは"|"でつなぐ

条件式は必ず()で括らないと意図した評価順にならず、`TypeError`となる

※ pandasと異なり、query関数は存在しない

In [5]:
df_users.filter((pl.col("type") == "enterprise") | (pl.col("leaved") == True))

id,name,email,type,leaved
str,str,str,str,bool
"""AQA7LkexXv""","""チャーリー""","""foo@foo.net""","""personal""",true
"""C82ZQKSQk7""","""ダービー""","""baz@baz.biz""","""enterprise""",false
"""HzZow64HGH""","""エリザベス""","""bar@bar.info""","""enterprise""",false


※ polars v0.14.0以降、DataFrameのattributeによる特定列を指定は廃止となった。特定列の参照だけであれば`polars.DataFrame.get_column`および`polars.DataFrame.get_columns`、特定列の更新であれば`polars.DataFrame.with_column`および`polars.DataFrame.with_columns`を使う。これら処理は並列に実行されるので、polarsにおける速度改善の面でも推奨される。

公式ドキュメント: https://pola-rs.github.io/polars-book/user-guide/coming_from_pandas.html#column-assignment

## ユーザ数を出す

### 全ユーザ数

In [6]:
len(df_users)

5

### leavedがFalseであるユーザの数(入会ユーザ数)

In [7]:
len(df_users.filter(~pl.col("leaved")))

4

### メールアドレスが"b"から始まっているユーザを取りだし: polars.Series.apply

今回のケースでは、df_usersのemailだけ参照すればいいので、df_users.get_column("email")にapplyを使う方法が望ましい(よりスループットが高く、必要なデータのみ触るため)。polarsでは、第二引数(return_dtype)で返り値の型を指定しておく必要がある。また、よりシンプルに `df_users.filter((pl.col("email") >= "b") & (pl.col("email") < "c"))`とすることも可。

なお、polars.DataFrame.applyではcallableに渡されるのはTuple型の各行データになっている。


In [8]:
serial_email_b_start = df_users.get_column("email").apply(lambda email: email.startswith("b"))
print(len(df_users.filter(serial_email_b_start)))

2


## ユニークなユーザタイプをリストアップし、カウントアップ: polars.DataFrame.unique, polars.Series.to_list

- pandas.DataFrame.drop_duplicates ≒ polars.DataFrame.unique
- pandas.Series.to_list = polars.Series.to_list

In [9]:
user_types = df_users.unique(subset="type").get_column("type").to_list()
user_types

['personal', 'enterprise']

In [10]:
{
    user_type: len(df_users.filter(pl.col("type") == user_type))
    for user_type in user_types
} 

{'personal': 3, 'enterprise': 2}

# ユーザによる操作データの解析

id="OoXNK4b3px"について、actions.csv内でpointが空なのでnullになる

主なデータは整数値のため、pointはi64になっている(pandasではNaN=float。暗黙の型変換でpointがfloatになる)

In [11]:
df_actions = pl.read_csv("actions.csv")
df_actions

id,user_id,date,message,point
str,str,str,str,i64
"""7GsubTX9n6""","""BcHgeZkTsc""","""2021-08-15T10:...","""Hoge hoge""",1
"""D76FJVQ2j2""","""KHPiabVr3o""","""2021-08-15T10:...","""Lorem Ipsum""",21
"""6znyhCukd6""","""HzZow64HGH""","""2021-08-15T11:...","""テスト テスト""",3
"""hSQszmDjlU""","""BcHgeZkTsc""","""2021-08-15T12:...","""テスト テスト２""",42
"""CVqQD0xH2Y""","""HzZow64HGH""","""2021-08-15T14:...","""👑👑💢""",49
"""OoXNK4b3px""","""BcHgeZkTsc""","""2021-08-15T14:...","""🔡""",null
"""veDQHBOXnG""","""BcHgeZkTsc""","""2021-08-15T14:...","""foo""",4


## 投稿データをuser_id昇順→point降順にする: sort

引数で渡すリストの各アイテムの順番がそれぞれ対応する(以下例であれば、user_idがreverse=False, pointがreverse=True)

(as of v0.16.14) reverse属性は廃止され、descending属性を使う形になっている。

(as of v0.15.8) pandas.DataFrame.sort_valuesはpolars.DataFrame.sortに対応する。acsending属性はreverse属性に対応するが、acsendingはTrueで昇順、reverseはTrueで降順になっている。


In [12]:
sort_orders = [False, True]
df_actions.sort(["user_id", "point"], descending=sort_orders)

id,user_id,date,message,point
str,str,str,str,i64
"""hSQszmDjlU""","""BcHgeZkTsc""","""2021-08-15T12:...","""テスト テスト２""",42
"""veDQHBOXnG""","""BcHgeZkTsc""","""2021-08-15T14:...","""foo""",4
"""7GsubTX9n6""","""BcHgeZkTsc""","""2021-08-15T10:...","""Hoge hoge""",1
"""OoXNK4b3px""","""BcHgeZkTsc""","""2021-08-15T14:...","""🔡""",null
"""CVqQD0xH2Y""","""HzZow64HGH""","""2021-08-15T14:...","""👑👑💢""",49
"""6znyhCukd6""","""HzZow64HGH""","""2021-08-15T11:...","""テスト テスト""",3
"""D76FJVQ2j2""","""KHPiabVr3o""","""2021-08-15T10:...","""Lorem Ipsum""",21


## pointの値なし(null)を置き換える: fill_null

valueで更新したい値を指定するか、strategyで一定ルールで値を埋める。

fill_nullのstrategy属性に指定できるものは以下。

- "forward": 直前の値で置き換える
- "backward": 直後の値で置き換える
- "mean": 平均値で置き換える
- "max": 最大値で置き換える
- "min": 最小値で置き換える
- "one": 1で置き換える
- "zero": 0で置き換える

(as of v0.16.4) with_column属性は廃止されたので、with_columns属性を代わりに使用する。

In [13]:
df_actions = df_actions.with_columns(
    pl.col("point").fill_null(strategy="zero")
)
df_actions[["id", "point"]]

id,point
str,i64
"""7GsubTX9n6""",1
"""D76FJVQ2j2""",21
"""6znyhCukd6""",3
"""hSQszmDjlU""",42
"""CVqQD0xH2Y""",49
"""OoXNK4b3px""",0
"""veDQHBOXnG""",4


## ユーザごとにpointを合算: groupby

groupbyの引数で指定した"user_id"はIndexに変換される

aggにて、どのパラメータをどの関数・処理で集計するかを指定する(指定しないパラメータは出力されない)。pandasの場合と異なり、dictで各列に関数を渡す形ではなく、リストで列選択＋処理内容を渡すイメージ。

(as of v140) 集約後の結果について昇順ソートされなくなっている。

In [14]:
df_actions \
    .groupby(["user_id"]) \
    .agg([
         pl.col("point").sum()
    ])

user_id,point
str,i64
"""BcHgeZkTsc""",47
"""HzZow64HGH""",52
"""KHPiabVr3o""",21


## ユーザごとのpointの統計値を出す: describe
pandasでは

- count, mean,std,min,25%,50%,75%,max

が出力されるが、polarsでは

- mean, std, min, median(=50%), max

が出力される。count, 25%, 75%に相当する統計値は出さない。

In [15]:
dfs = []
for user_id in df_actions["user_id"]:
    df = df_actions.filter(pl.col("user_id") == user_id)[["point"]].describe()
    df = df.with_columns(pl.Series(name="user_id", values=pl.Series([user_id * len(df)])))
    dfs.append(df)
pl.concat(dfs).pivot(values="point", index="user_id", columns="describe")

user_id,count,null_count,mean,std,min,max,median
str,f64,f64,f64,f64,f64,f64,f64
"""BcHgeZkTscBcHg...",4.0,0.0,11.75,20.238165,0.0,42.0,2.5
"""KHPiabVr3oKHPi...",1.0,0.0,21.0,0.0,21.0,21.0,21.0
"""HzZow64HGHHzZo...",2.0,0.0,26.0,32.526912,3.0,49.0,26.0


## ユーザごとに初めてのaction取り出し: groupby

In [16]:
df_actions \
    .groupby(["user_id"]) \
    .agg([
        pl.col("date").first(),
        pl.col("message").first(),
        pl.col("point").first(),
    ])

user_id,date,message,point
str,str,str,i64
"""BcHgeZkTsc""","""2021-08-15T10:...","""Hoge hoge""",1
"""HzZow64HGH""","""2021-08-15T11:...","""テスト テスト""",3
"""KHPiabVr3o""","""2021-08-15T10:...","""Lorem Ipsum""",21


## 10分ごとのpoint値の小計を出力: groupby_dynamic
groupby_dynamic + sumにより、10分ごとの区間でpointの小計を取る。

-5Minのoffsetを入れることで、最初action日時=2021-08-15T10:12:34+09:00を含む区間として10:05(=10:10 - 0:05)から集計していく

※ v0.14.19でdatetimeのタイムゾーン周りの処理が改善され、print時にタイムゾーンが表示されるようになっている。



In [17]:
df_actions_date = df_actions.clone()
df_actions_date = df_actions_date.with_columns(
    pl.col("date").str.strptime(
        pl.Datetime, "%Y-%m-%dT%H:%M:%S%z"
    )
)
df_actions_date[["date", "point"]] \
    .groupby_dynamic(
        "date", 
        every="10m", 
        offset="-5m"    # FIXME: offset may not work?
    ) \
    .agg([
        pl.col("point").sum(),
    ])

date,point
"datetime[μs, +09:00]",i64
2021-08-15 10:05:00 +09:00,1
2021-08-15 10:15:00 +09:00,21
2021-08-15 11:45:00 +09:00,3
2021-08-15 12:25:00 +09:00,42
2021-08-15 14:35:00 +09:00,49
2021-08-15 14:45:00 +09:00,4


## actionに対するリアクションデータの解析

In [18]:
df_reactions_1 = pl.read_csv("reactions_1.csv")
df_reactions_1

id,action_id,user_id,date,type,reward
str,str,str,str,str,i64
"""ycV0zbqrL5""","""7GsubTX9n6""","""AQA7LkexXv""","""2021-08-15T11:...","""like""",1
"""NXXk7iEsMA""","""OoXNK4b3px""","""AQA7LkexXv""","""2021-08-15T13:...","""like""",1
"""Oq9i6DBTBp""","""OoXNK4b3px""","""AQA7LkexXv""","""2021-08-15T13:...","""like""",1


In [19]:
df_reactions_2 = pl.read_csv("reactions_2.csv")
df_reactions_2

id,action_id,user_id,date,type,reward
str,str,str,str,str,i64
"""jwUcUCuVEq""","""D76FJVQ2j2""","""BcHgeZkTsc""","""2021-08-15T16:...","""comment""",3


## 2データを1つのDataFrameに結合する: concat

In [20]:
df_reactions = pl.concat([df_reactions_1, df_reactions_2])
df_reactions

id,action_id,user_id,date,type,reward
str,str,str,str,str,i64
"""ycV0zbqrL5""","""7GsubTX9n6""","""AQA7LkexXv""","""2021-08-15T11:...","""like""",1
"""NXXk7iEsMA""","""OoXNK4b3px""","""AQA7LkexXv""","""2021-08-15T13:...","""like""",1
"""Oq9i6DBTBp""","""OoXNK4b3px""","""AQA7LkexXv""","""2021-08-15T13:...","""like""",1
"""jwUcUCuVEq""","""D76FJVQ2j2""","""BcHgeZkTsc""","""2021-08-15T16:...","""comment""",3


# 複数データをmergeして解析する

## action, reactionデータをactionのidで左結合: join, (rename)
polars.DataFrame.mergeは存在しない。代わりに、polars.DataFrame.joinで対応する。

polars.DataFrame.renameは、DataFrameにindexがないのでcolumnsを指定する必要がない。


In [21]:
df_reactions_renamed = df_reactions.rename({
    "id": "reaction_id", 
    "user_id": "reaction_user_id",
    "date": "reaction_date",
})
df_actions_renamed = df_actions.rename({"id": "action_id"})

df_actions_merged = df_actions_renamed.join(
    df_reactions_renamed,
    on=["action_id"],
    how="left"
)

df_actions_merged = df_actions_merged.with_columns(
    pl.col("reward").fill_null(value=0)
)
df_actions_merged

action_id,user_id,date,message,point,reaction_id,reaction_user_id,reaction_date,type,reward
str,str,str,str,i64,str,str,str,str,i64
"""7GsubTX9n6""","""BcHgeZkTsc""","""2021-08-15T10:...","""Hoge hoge""",1,"""ycV0zbqrL5""","""AQA7LkexXv""","""2021-08-15T11:...","""like""",1
"""D76FJVQ2j2""","""KHPiabVr3o""","""2021-08-15T10:...","""Lorem Ipsum""",21,"""jwUcUCuVEq""","""BcHgeZkTsc""","""2021-08-15T16:...","""comment""",3
"""6znyhCukd6""","""HzZow64HGH""","""2021-08-15T11:...","""テスト テスト""",3,null,null,null,null,0
"""hSQszmDjlU""","""BcHgeZkTsc""","""2021-08-15T12:...","""テスト テスト２""",42,null,null,null,null,0
"""CVqQD0xH2Y""","""HzZow64HGH""","""2021-08-15T14:...","""👑👑💢""",49,null,null,null,null,0
"""OoXNK4b3px""","""BcHgeZkTsc""","""2021-08-15T14:...","""🔡""",0,"""NXXk7iEsMA""","""AQA7LkexXv""","""2021-08-15T13:...","""like""",1
"""OoXNK4b3px""","""BcHgeZkTsc""","""2021-08-15T14:...","""🔡""",0,"""Oq9i6DBTBp""","""AQA7LkexXv""","""2021-08-15T13:...","""like""",1
"""veDQHBOXnG""","""BcHgeZkTsc""","""2021-08-15T14:...","""foo""",4,null,null,null,null,0


## 上記データにユーザ名を追加: join
df_usersにあるemail, type, leavedは以後使わない

そのため、"id", "name"列以外をdropしている(下記コードの1行目)

df_actions_mergedにあるuser_idのみをmerge対象にしたいので、左結合(df_actions_merged.user_idを基準にマージ)する

In [22]:
df_users_dropped = df_users[["id", "name"]]
df_merged = df_actions_merged.join(
    df_users_dropped.rename({"id": "user_id"}),
    left_on=["user_id"],
    right_on=["user_id"],
    how="left")
df_merged

action_id,user_id,date,message,point,reaction_id,reaction_user_id,reaction_date,type,reward,name
str,str,str,str,i64,str,str,str,str,i64,str
"""7GsubTX9n6""","""BcHgeZkTsc""","""2021-08-15T10:...","""Hoge hoge""",1,"""ycV0zbqrL5""","""AQA7LkexXv""","""2021-08-15T11:...","""like""",1,"""アリス"""
"""D76FJVQ2j2""","""KHPiabVr3o""","""2021-08-15T10:...","""Lorem Ipsum""",21,"""jwUcUCuVEq""","""BcHgeZkTsc""","""2021-08-15T16:...","""comment""",3,"""ボブ"""
"""6znyhCukd6""","""HzZow64HGH""","""2021-08-15T11:...","""テスト テスト""",3,null,null,null,null,0,"""エリザベス"""
"""hSQszmDjlU""","""BcHgeZkTsc""","""2021-08-15T12:...","""テスト テスト２""",42,null,null,null,null,0,"""アリス"""
"""CVqQD0xH2Y""","""HzZow64HGH""","""2021-08-15T14:...","""👑👑💢""",49,null,null,null,null,0,"""エリザベス"""
"""OoXNK4b3px""","""BcHgeZkTsc""","""2021-08-15T14:...","""🔡""",0,"""NXXk7iEsMA""","""AQA7LkexXv""","""2021-08-15T13:...","""like""",1,"""アリス"""
"""OoXNK4b3px""","""BcHgeZkTsc""","""2021-08-15T14:...","""🔡""",0,"""Oq9i6DBTBp""","""AQA7LkexXv""","""2021-08-15T13:...","""like""",1,"""アリス"""
"""veDQHBOXnG""","""BcHgeZkTsc""","""2021-08-15T14:...","""foo""",4,null,null,null,null,0,"""アリス"""


## ユーザごとの活動量(point + reward)を計算

In [23]:
df_sum = df_merged \
    .groupby("user_id") \
    .agg([
        pl.col("point").fill_null(0).sum() + pl.col("reward").fill_null(0).sum(),
    ])
df_sum = df_sum.rename({"point": "activity"})
df_sum

user_id,activity
str,i64
"""KHPiabVr3o""",24
"""HzZow64HGH""",52
"""BcHgeZkTsc""",50


## ユーザごとに活動量のランキングを出す

In [24]:
df_ranking = df_sum \
    .join(
        df_users_dropped.rename({"id": "user_id"}),
        on="user_id",
        how="left"
    ) \
    .sort(["activity"], descending=True)
df_ranking

user_id,activity,name
str,i64,str
"""HzZow64HGH""",52,"""エリザベス"""
"""BcHgeZkTsc""",50,"""アリス"""
"""KHPiabVr3o""",24,"""ボブ"""
